In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1668662209269_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.conf import SparkConf
from pyspark.sql.functions import sha2, concat_ws
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
conf = SparkConf() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Fetching the app config path
conf.get("spark.path")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://shibi-lz-07/configuration/app-config.json'

In [38]:
# Converting it into the spark dataframe
appconfig_df=spark.read.option("multiline","true").format("JSON").load(conf.get("spark.path"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# Got the individual path respectively
appconfig_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|             actives|       final-actives|    final-viewership|          viewership|
+--------------------+--------------------+--------------------+--------------------+
|[[s3://shibi-rz-0...|[[s3://shibi-sz-0...|[[s3://shibi-sz-0...|[[s3://shibi-rz-0...|
+--------------------+--------------------+--------------------+--------------------+

In [73]:
class SparkJob:
    def __init__(self,rawactive,rawviewership):
        self.rawactive=rawactive
        self.rawviewership=rawviewership
    def find(path):
        
    def readRawzone(self):
        raw_actives=spark.read.option("header",True).option("inferSchema",True).parquet(self.rawactive)
        raw_viewership=spark.read.option("header",True).option("inferSchema",True).parquet(self.rawviewership)
        return raw_actives,raw_viewership
    def maskingcolumns(self,active_df,viewership_df,column1,column2):
        masked_actives=active_df.withColumn("masked_advertising_id", sha2(active_df[column1], 256))
        masked_actives=masked_actives.withColumn("masked_user_id", sha2(masked_actives[column2], 256))
        masked_viewership=viewership_df.withColumn("masking_advertise_id", sha2(concat_ws("||", viewership_df[column1]), 256))
        return masked_actives,masked_viewership
    def transformDecimalPrecison(self,active_df,viewership_df,column1,column2,column3,column4,precision):
        transformed_actives=active_df.withColumn("user_latitude", active_df[column1].cast(DecimalType(precision)))
        transformed_actives=transformed_actives.withColumn("user_longitude", transformed_actives[column2].cast(DecimalType(precision)))
        transformed_viewership=viewership_df.withColumn("user_lat", viewership_df[column3].cast(DecimalType(precision)))
        transformed_viewership=transformed_viewership.withColumn("user_long", transformed_viewership[column4].cast(DecimalType(precision)))
        return transformed_actives,transformed_viewership
    def Partitionandstagingzone(self,active_df,viewership_df,actives_sz_path,viewership_sz_path):
        active_df.withColumn("year", year(col("date"))).withColumn("day", dayofmonth(col("date"))).write.partitionBy('year','month','day').mode('Overwrite').parquet(actives_sz_path)
        viewership_df.withColumn("year", year(col("date"))).withColumn("day", dayofmonth(col("date"))).write.partitionBy('year','month','day').mode('Overwrite').parquet(viewership_sz_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
for i in range(5):
    print(appconfig_df.first()['final-actives'][i])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(data-location='s3://shibi-sz-07/actives', file-format='parquet')
['advertising_id', 'user_id']
['month', 'date']
Row(data-location='s3://shibi-rz-07/actives', file-format='parquet')
Row(cols=['user_latitude', 'user_longitude', 'location_source'], precision='7')

In [77]:
# appconfig_df.first()['final-viewership'][4][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['user_lat', 'user_long', 'location_source']

In [78]:
# appconfig_df.first()['final-actives'][0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://shibi-sz-07/actives'

In [79]:
# appconfig_df.first()['actives'][1][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://shibi-lz-07/Actives.parquet.snappy'

In [80]:
obj=SparkJob(appconfig_df.first()['actives'][1][0],appconfig_df.first()['viewership'][1][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
active,viewership=obj.readRawzone()
active,viewership=obj.maskingcolumns(active,viewership,appconfig_df.first()['final-actives'][1][0],appconfig_df.first()['final-actives'][1][1])
active,viewership=obj.transformDecimalPrecison(active,viewership,appconfig_df.first()['final-actives'][4][0][0],appconfig_df.first()['final-actives'][4][0][1],appconfig_df.first()['final-viewership'][4][0][0],appconfig_df.first()['final-viewership'][4][0][1],int(appconfig_df.first()['final-viewership'][4][1]))
obj.Partitionandstagingzone(active,viewership,appconfig_df.first()['final-actives'][0][0],appconfig_df.first()['final-viewership'][0][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…